In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,f1_score,recall_score,precision_score,accuracy_score,precision_recall_curve,roc_curve,roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from collections import Counter
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dropout
from keras import regularizers
import warnings
warnings.filterwarnings('ignore')
import random
random.seed(0)

Using TensorFlow backend.


In [37]:
dataset = pd.read_csv("/home/hr/Documents/creditcard.csv")
dataset1 = dataset.ix[:,1:-1]
print(dataset1.shape)
print(dataset.shape)
X = dataset.iloc[:,1:-1]
Y = dataset.iloc[:,-1]

(284807, 29)
(284807, 31)


In [28]:
from imblearn.ensemble import BalanceCascade
from imblearn.tensorflow import balanced_batch_generator
from imblearn.datasets import make_imbalance
class_dict = dict()
from collections import Counter
from sklearn.linear_model import LogisticRegression
X1, y1 = make_imbalance(X, Y, class_dict)
X1 = X1.astype(np.float32)
batch_size, learning_rate, epochs = 10000, 0.01, 20
training_generator, steps_per_epoch = balanced_batch_generator(X1, y1, sample_weight=None, sampler=None,
                                                               batch_size=batch_size, random_state=42)

In [29]:
import tensorflow as tf
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))
def accuracy(y_true, y_pred):
    return np.mean(np.argmax(y_pred, axis=1) == y_true)

In [32]:
input_size, output_size = X1.shape[1], 2
data = tf.placeholder("float32", shape=[None, input_size])
targets = tf.placeholder("int32", shape=[None])
# build the model and weights
W = init_weights([input_size, output_size])
b = init_weights([output_size])
out_act = tf.nn.sigmoid(tf.matmul(data, W) + b)
# build the loss, predict, and train operator
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=out_act, labels=targets)
loss = tf.reduce_sum(cross_entropy)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss)
predict = tf.nn.softmax(out_act)
# Initialization of all variables in the graph
init = tf.global_variables_initializer()
with tf.Session() as sess:
    print('Starting training')
    sess.run(init)
    for e in range(epochs):
        for i in range(steps_per_epoch):
            X_batch, y_batch = next(training_generator)
            feed_dict = dict()
            feed_dict[data] = X_batch; feed_dict[targets] = y_batch
            sess.run([train_op, loss], feed_dict=feed_dict)
            # For each epoch, run accuracy on train and test
        feed_dict = dict()
        feed_dict[data] = X1
        predicts_train = sess.run(predict, feed_dict=feed_dict)
        print("epoch: {} train accuracy: {:.3f} " .format(e, accuracy(y1, predicts_train)))
            # doctest: +ELLIPSIS

Starting training
epoch: 0 train accuracy: 0.968 
epoch: 1 train accuracy: 0.968 
epoch: 2 train accuracy: 0.968 
epoch: 3 train accuracy: 0.968 
epoch: 4 train accuracy: 0.968 
epoch: 5 train accuracy: 0.968 
epoch: 6 train accuracy: 0.968 
epoch: 7 train accuracy: 0.968 
epoch: 8 train accuracy: 0.968 
epoch: 9 train accuracy: 0.968 
epoch: 10 train accuracy: 0.968 
epoch: 11 train accuracy: 0.968 
epoch: 12 train accuracy: 0.968 
epoch: 13 train accuracy: 0.968 
epoch: 14 train accuracy: 0.968 
epoch: 15 train accuracy: 0.968 
epoch: 16 train accuracy: 0.968 
epoch: 17 train accuracy: 0.968 
epoch: 18 train accuracy: 0.968 
epoch: 19 train accuracy: 0.968 
